In [ ]:
!pip install deep-translator


In [ ]:
!pip install gensim

In [ ]:
!pip install langdetect


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings
from gensim import corpora, models, similarities
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import pandas as pd
import re
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from deep_translator import GoogleTranslator
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import pandas as pd
import time


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/MyDrive/7._Do_an_tot_nghiep/Project_1"

'Content-Base Similarity.ipynb'
 Data
 data_for_classification.csv
 data_for_classification_pre.csv
 data_with_topic.csv
 Demo
 demo_project_2
'ITviec Job Reviews Sentiment Analyst and Text Clustering.ipynb'
'Note phân tích.gdoc'
 pipeline_like.pkl
'Previous_version_(to_delete)'
 Project_2_Recommend_Classification_Pyspark.ipynb
 Project_Overview
 Reviews_cleaned_for_SA_v2.csv
 Reviews_cleaned_for_TC_v2.csv
 Reviews_translated.csv
 rf_pipeline_model_v2.joblib
'Sentiment Analysis v2.ipynb'
 Slide_project_1.pptx
 Slide_project_2.pptx
 Support_file


In [ ]:
 %cd "/content/gdrive/MyDrive/7._Do_an_tot_nghiep/Project_1"

/content/gdrive/.shortcut-targets-by-id/1HLhEWyfTcJA3kkYm4gKnF38JiKaX9vvv/7._Do_an_tot_nghiep/Project_1


#EDA Dataset

In [ ]:
# Đọc file Excel
df = pd.read_excel("Data/Overview_Companies.xlsx")

# Xem 5 dòng đầu tiên
df.head()

,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href
0,0,1BITLAB Technology,IT Outsourcing,Software Development Outsourcing,1-50 employees,Vietnam,Monday - Friday,No OT,1BITLAB Technology\nCông ty cổ phần công nghệ ...,NaN,NaN,"{'Ha Noi': ['81 Lê Đức Thọ, Nam Từ LIêm, Hà Nộ...",https://itviec.com/companies/1bitlab-technology\n
1,1,1test,IT Product,IT Services and IT Consulting,1-50 employees,South Korea,Monday - Friday,No OT,1test\nIs an innovative educational technology...,NaN,NaN,"{'Ho Chi Minh': ['Remote, District 1, Ho Chi M...",https://itviec.com/companies/1test\n
2,2,52 Entertainment,IT Product,Game,1-50 employees,France,Monday - Friday,No OT,A start-up of French and Vietnamese guys with ...,Our Key Skills\nJava\nGames\nQA QC\nC#\nUnity\...,Be your own boss\nA free lunch and a half day ...,"{'Ho Chi Minh': ['Tầng 7, 23 Trần Cao Vân, Phư...",https://itviec.com/companies/52-entertainment\n
3,3,8Seneca,IT Outsourcing,IT Services and IT Consulting,1-50 employees,Singapore,Monday - Friday,No OT,Pure play IT team extensions B2b exclusive out...,NaN,"Ha Noi\n19F, Coninco Tower 4 Tôn Thất Tùng, Do...","{'Ha Noi': ['19F, Coninco Tower 4 Tôn Thất Tùn...",https://itviec.com/companies/8seneca\n
4,4,Accenture,IT Service and IT Consulting,IT Services and IT Consulting,301-500 employees,Vietnam,Monday - Friday,No OT,A global professional services company with le...,Our Key Skills\nCloud\nLinux\nDevOps\nNetworki...,Hybrid working\nWonderful and Human focus Envi...,"{'Ho Chi Minh': ['9th Floor, Viettel Tower, 28...",https://itviec.com/companies/accenture\n


In [ ]:
# Thống kê mô tả với các cột dạng object và số
df.describe(include="all")


,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href
count,478.000000,478,478,463,478,478,476,447,478,321,379,478,478
unique,NaN,478,4,33,6,26,2,4,477,321,379,475,478
top,NaN,1BITLAB Technology,IT Product,Software Products and Web Services,1-50 employees,Vietnam,Monday - Friday,No OT,"CBTW is a global tech company with 3,000+ prof...",Our Key Skills\nJava\nGames\nQA QC\nC#\nUnity\...,Be your own boss\nA free lunch and a half day ...,"{'Ho Chi Minh': ['CII Building, 152 Điên Biên ...",https://itviec.com/companies/1bitlab-technology\n
freq,NaN,1,295,107,178,259,449,389,2,1,1,2,1
mean,238.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,138.130976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,119.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,238.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,357.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Kiểm tra số lượng giá trị thiếu trên từng cột
df.isnull().sum()


,0
id,0
Company Name,0
Company Type,0
Company industry,15
Company size,0
Country,0
Working days,2
Overtime Policy,31
Company overview,0
Our key skills,157


In [ ]:
# Kiểm tra xem có cột nào kiểu dữ liệu không đúng mong muốn không
df.dtypes


,0
id,int64
Company Name,object
Company Type,object
Company industry,object
Company size,object
Country,object
Working days,object
Overtime Policy,object
Company overview,object
Our key skills,object


In [ ]:
# Hàm detect ngôn ngữ an toàn (tránh lỗi khi ô trống)
def detect_language(text):
    try:
        if pd.isna(text) or str(text).strip() == "":
            return "unknown"
        return detect(text)
    except LangDetectException:
        return "error"

# Danh sách cột cần kiểm tra
columns_to_check = [
    "Company overview",
    "Our key skills",
    "Why you'll love working here"
]

# Áp dụng detect vào từng dòng trong từng cột
for col in columns_to_check:
    lang_col = f"{col}_lang"
    df[lang_col] = df[col].apply(detect_language)

# Đếm số lượng ngôn ngữ trong mỗi cột
for col in columns_to_check:
    lang_col = f"{col}_lang"
    print(f"\n🔍 Language counts in column: '{col}':")
    print(df[lang_col].value_counts())



🔍 Language counts in column: 'Company overview':
Company overview_lang
en       324
vi       149
de         2
error      2
fi         1
Name: count, dtype: int64

🔍 Language counts in column: 'Our key skills':
Our key skills_lang
en         266
unknown    157
sv          15
ca           8
vi           7
de           5
et           5
tl           4
no           4
ro           2
it           1
so           1
hr           1
cy           1
af           1
Name: count, dtype: int64

🔍 Language counts in column: 'Why you'll love working here':
Why you'll love working here_lang
en         261
vi         114
unknown     99
error        1
de           1
id           1
fr           1
Name: count, dtype: int64


#Phân lớn lă tiếng anh nên sử dụng tiếng anh là chính

In [ ]:
# Hàm detect ngôn ngữ an toàn
def detect_language(text):
    try:
        if pd.isna(text) or str(text).strip() == "":
            return "unknown"
        return detect(text)
    except LangDetectException:
        return "unknown"

# Hàm dịch tiếng Việt sang tiếng Anh nếu cần
def translate_if_needed(text):
    try:
        if pd.isna(text) or str(text).strip() == "":
            return ""
        lang = detect_language(text)
        if lang == "vi":
            return GoogleTranslator(source='vi', target='en').translate(text)
        else:
            return text  # không dịch nếu đã là tiếng Anh
    except Exception as e:
        print("Lỗi dịch:", e)
        return text  # fallback giữ nguyên


In [ ]:
columns_to_translate = [
    "Company overview",
    "Our key skills",
    "Why you'll love working here"
]

for col in columns_to_translate:
    print(f"🔄 Translating column: {col}")
    df[col] = df[col].apply(translate_if_needed)
    time.sleep(1)  # để tránh rate limit khi dịch nhiều dòng liên tục


🔄 Translating column: Company overview
🔄 Translating column: Our key skills
🔄 Translating column: Why you'll love working here


In [ ]:
df[[
    "Company overview",
    "Our key skills",
    "Why you'll love working here"
]].sample(5, random_state=42)


#Làm sạch các cột

In [ ]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


In [ ]:
custom_stopwords = [
    # Articles
    "a", "an", "the",
    # Prepositions
    "in", "on", "at", "to", "from", "by", "of", "with",
    # Conjunctions
    "and", "but", "or", "for", "nor", "so", "yet",
    # Pronouns
    "i", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them", "we’re",
    # Auxiliary verbs
    "be", "have", "do", "does", "did", "was", "were", "will", "would", "shall", "should", "may", "might", "can", "could", "must",
    # Other common words
    "that", "this", "which", "what", "their", "these", "those",
    # Web junk
    "https", "www"
]


In [ ]:


# Kết hợp stopwords
combined_stopwords = set(ENGLISH_STOP_WORDS).union(set(custom_stopwords))


In [19]:


def clean_text_remove_combined_stopwords(text):
    if pd.isna(text):
        return ""

    # Lowercase + giữ lại các từ a-z
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())

    # Loại bỏ tất cả stopwords kết hợp
    filtered_words = [word for word in words if word not in combined_stopwords]

    return ' '.join(filtered_words)


In [20]:
columns_to_clean = [
    "Company overview",
    "Our key skills",
    "Why you'll love working here"
]

for col in columns_to_clean:
    clean_col = f"{col}_clean"
    df[clean_col] = df[col].apply(clean_text_remove_combined_stopwords)


In [21]:
index = 10  # bạn có thể đổi sang dòng khác để xem

pd.DataFrame({
    "Overview - Raw": df["Company overview"].iloc[index],
    "Overview - Clean": df["Company overview_clean"].iloc[index],
    "Skills - Raw": df["Our key skills"].iloc[index],
    "Skills - Clean": df["Our key skills_clean"].iloc[index],
    "Love Here - Raw": df["Why you'll love working here"].iloc[index],
    "Love Here - Clean": df["Why you'll love working here_clean"].iloc[index]
}, index=[0])


,Overview - Raw,Overview - Clean,Skills - Raw,Skills - Clean,Love Here - Raw,Love Here - Clean
0,ALLEXCEED VIETNAM\nALLEXCEED VIETNAM (founding...,allexceed vietnam allexceed vietnam founding s...,Our Key Skills\nJava\nJavaScript\n.NET\nPHP\ni...,key skills java javascript net php ios android...,Working opportunities (short/long -term) and J...,working opportunities short long term japanese...


In [22]:
def merge_unique_words(*columns):
    # Gộp các cột văn bản làm sạch → danh sách từ
    words = " ".join(columns).split()
    # Loại bỏ từ trùng nhau, giữ thứ tự ban đầu
    seen = set()
    unique_words = [word for word in words if not (word in seen or seen.add(word))]
    return " ".join(unique_words)

# Tạo cột full_text từ 3 cột clean
df["full_text"] = df.apply(
    lambda row: merge_unique_words(
        row["Company overview_clean"],
        row["Our key skills_clean"],
        row["Why you'll love working here_clean"]
    ),
    axis=1
)

In [23]:
df[["Company overview_clean", "Our key skills_clean", "Why you'll love working here_clean", "full_text"]].sample(3, random_state=7)


,Company overview_clean,Our key skills_clean,Why you'll love working here_clean,full_text
138,largest vertically integrated media entertainm...,golang devops engineer security,,largest vertically integrated media entertainm...
284,dev innovation technological advancement aim p...,,ho chi minh hb tower dien bien phu ward binh t...,dev innovation technological advancement aim p...
340,pasona vietnam specializes providing software ...,key skills sap javascript sql japanese erp oracle,high promotion opportunities friendly dynamic ...,pasona vietnam specializes providing software ...


In [24]:
from gensim import corpora, models, similarities
df["tokens"] = df["full_text"].apply(lambda x: x.split())


In [25]:
# Dictionary: ánh xạ từ → id
dictionary = corpora.Dictionary(df["tokens"])

# Corpus: mỗi văn bản → danh sách (token_id, count)
corpus = [dictionary.doc2bow(text) for text in df["tokens"]]


In [26]:
dictionary.token2id

{'activity': 0,
 'addition': 1,
 'ai': 2,
 'approaching': 3,
 'blockchain': 4,
 'career': 5,
 'changing': 6,
 'company': 7,
 'constantly': 8,
 'cooperation': 9,
 'customers': 10,
 'development': 11,
 'direction': 12,
 'directly': 13,
 'enhance': 14,
 'enthusiastic': 15,
 'environment': 16,
 'fast': 17,
 'field': 18,
 'japanese': 19,
 'joint': 20,
 'knowledge': 21,
 'major': 22,
 'management': 23,
 'market': 24,
 'opportunity': 25,
 'partners': 26,
 'professional': 27,
 'providing': 28,
 'regarding': 29,
 'specialized': 30,
 'specializes': 31,
 'stock': 32,
 'sustainable': 33,
 'team': 34,
 'technology': 35,
 'working': 36,
 'works': 37,
 'world': 38,
 'young': 39,
 'academies': 40,
 'advanced': 41,
 'artificial': 42,
 'assignments': 43,
 'committed': 44,
 'comprehensive': 45,
 'conduct': 46,
 'cutting': 47,
 'edge': 48,
 'education': 49,
 'educational': 50,
 'educators': 51,
 'elevate': 52,
 'empower': 53,
 'engagement': 54,
 'experience': 55,
 'experiences': 56,
 'gamified': 57,
 'gro

In [27]:
corpus[2] # id, so lan xuat hien cua token trong van ban/ san pham

[(11, 1),
 (34, 1),
 (36, 1),
 (67, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 1),
 (102, 1),
 (103, 1),
 (104, 1),
 (105, 1),
 (106, 1),
 (107, 1),
 (108, 1),
 (109, 1),
 (110, 1),
 (111, 1),
 (112, 1),
 (113, 1),
 (114, 1),
 (115, 1),
 (116, 1),
 (117, 1),
 (118, 1),
 (119, 1),
 (120, 1),
 (121, 1),
 (122, 1),
 (123, 1),
 (124, 1),
 (125, 1),
 (126, 1),
 (127, 1),
 (128, 1),
 (129, 1),
 (130, 1),
 (131, 1),
 (132, 1),
 (133, 1),
 (134, 1),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 1),
 (139, 1),
 (140, 1),
 (141, 1),
 (142, 1),
 (143, 1),
 (144, 1),
 (145, 1),
 (146, 1),
 (147, 1),
 (148, 1),
 (149, 1),
 (150, 1),
 (151, 1),
 (152, 1),
 (153, 1),
 (154, 1),
 (155, 1),
 (156, 1)]

In [39]:
# Full updated code including all corrected and unified components

import pandas as pd
import numpy as np
from gensim import corpora, models
from gensim.models import LsiModel, LdaModel, Word2Vec
from gensim.similarities import MatrixSimilarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF similarity
def compute_tfidf_similarity(texts, query_index):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    cosine_sim = cosine_similarity(tfidf_matrix[query_index], tfidf_matrix).flatten()
    cosine_sim[query_index] = -1  # Remove self-match
    return cosine_sim

# LSI similarity
def compute_lsi_similarity(corpus, dictionary, query_index, num_topics=100):
    tfidf_model = models.TfidfModel(corpus)
    corpus_tfidf = tfidf_model[corpus]
    lsi_model = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)
    corpus_lsi = lsi_model[corpus]
    index = MatrixSimilarity(corpus_lsi)
    query_lsi = lsi_model[corpus[query_index]]
    sims = index[query_lsi]
    sims[query_index] = -1  # Remove self-match
    return sims


# Unified model comparison function
def run_all_models_and_compare(df, text_column="full_text", company_index=0):
    texts = df[text_column].tolist()
    tokens = [text.split() for text in texts]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(text) for text in tokens]

    tfidf_sims = compute_tfidf_similarity(texts, company_index)
    lsi_sims = compute_lsi_similarity(corpus, dictionary, company_index)

    results = pd.DataFrame({
        "Company": df["Company Name"],
        "TF-IDF": tfidf_sims,
        "LSI": lsi_sims,
    })

    results["Best Match Model"] = results[["TF-IDF", "LSI"]].idxmax(axis=1)
    results["Max Similarity"] = results[["TF-IDF", "LSI"]].max(axis=1)
    return results.sort_values(by="Max Similarity", ascending=False)


In [43]:
comparison_df = run_all_models_and_compare(df, company_index=2)
comparison_df.head(10)


,Company,TF-IDF,LSI,Best Match Model,Max Similarity
179,Floware,0.134951,0.535219,LSI,0.535219
68,Cloud Technology,0.125517,0.507140,LSI,0.507140
387,Silicon Prime Labs,0.112490,0.494441,LSI,0.494441
121,Công ty TNHH Thankslab Việt Nam,0.079739,0.437506,LSI,0.437506
202,Goodia Việt Nam,0.107303,0.428684,LSI,0.428684
242,Jitsu Vietnam,0.092095,0.421072,LSI,0.421072
465,Vulcan Labs,0.065170,0.420718,LSI,0.420718
189,Gameloft,0.078474,0.419289,LSI,0.419289
474,Zen8Labs,0.083450,0.418904,LSI,0.418904
311,NEXON DEV VINA,0.102404,0.415928,LSI,0.415928


bold text#1. TF-IDF + Cosine Similarity

In [30]:
from gensim import models, similarities

# TF-IDF model
tfidf_model = models.TfidfModel(corpus)

# Biểu diễn mỗi văn bản dưới dạng TF-IDF vector
corpus_tfidf = tfidf_model[corpus]

# Ma trận similarity (cosine)
index_tfidf = similarities.MatrixSimilarity(corpus_tfidf, num_features=len(dictionary))


In [31]:
def recommend_similar_companies(company_index, top_k=5):
    # Bước 1: lấy vector TF-IDF của công ty cần tìm
    query_bow = corpus[company_index]
    query_tfidf = tfidf_model[query_bow]

    # Bước 2: tính độ tương đồng giữa công ty này và tất cả công ty khác
    sims = index_tfidf[query_tfidf]

    # Bước 3: sắp xếp theo độ tương đồng giảm dần
    sorted_sims = sorted(enumerate(sims), key=lambda x: -x[1])

    print(f"📍 Các công ty tương tự với công ty ở dòng {company_index} (Company Name: {df['Company Name'].iloc[company_index]}):\n")
    for idx, score in sorted_sims[1:top_k+1]:  # bỏ chính nó (ở vị trí đầu tiên)
        print(f"- [{score:.4f}] {df['Company Name'].iloc[idx]} (ID: {df['id'].iloc[idx]})")


In [32]:
recommend_similar_companies(company_index=2, top_k=5)


📍 Các công ty tương tự với công ty ở dòng 2 (Company Name: 52 Entertainment):

- [0.1122] Floware (ID: 179)
- [0.1046] Cloud Technology (ID: 68)
- [0.0862] MEMENT Vietnam (ID: 281)
- [0.0855] ANTADA (ID: 15)
- [0.0811] HayHay Global (ID: 209)


#Tạo pineline

In [33]:
# ----- TF-IDF + Cosine Similarity Pipeline (Full Version with Updates) -----

import pandas as pd
import numpy as np
import re
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# ---------- STEP 1: TEXT CLEANING + STOPWORDS ----------

custom_stopwords = {
    "a", "an", "the", "in", "on", "at", "to", "from", "by", "of", "with", "and",
    "but", "or", "for", "nor", "so", "yet", "i", "you", "he", "she", "it", "we",
    "they", "me", "him", "her", "us", "them", "we’re", "be", "have", "do", "does",
    "did", "was", "were", "will", "would", "shall", "should", "may", "might", "can",
    "could", "must", "that", "this", "which", "what", "their", "these", "those",
    "https", "www"
}
combined_stopwords = set(ENGLISH_STOP_WORDS).union(custom_stopwords)

def clean_text(text):
    if pd.isna(text):
        return ""
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    filtered = [w for w in words if w not in combined_stopwords]
    return ' '.join(filtered)

def merge_unique_words(*columns):
    words = " ".join(columns).split()
    seen = set()
    unique_words = [w for w in words if not (w in seen or seen.add(w))]
    return " ".join(unique_words)

# ---------- STEP 2: LOAD + PREPARE DATA ----------

def prepare_tfidf_pipeline(df):
    # Làm sạch các cột riêng biệt
    df["Company overview_clean"] = df["Company overview"].apply(clean_text)
    df["Our key skills_clean"] = df["Our key skills"].apply(clean_text)
    df["Why you'll love working here_clean"] = df["Why you'll love working here"].apply(clean_text)

    # Gộp và loại trùng từ
    df["full_text"] = df.apply(lambda row: merge_unique_words(
        row["Company overview_clean"],
        row["Our key skills_clean"],
        row["Why you'll love working here_clean"]
    ), axis=1)

    # Tokenize
    df["tokens"] = df["full_text"].apply(lambda x: x.split())

    # Dictionary & Corpus
    dictionary = corpora.Dictionary(df["tokens"])
    corpus = [dictionary.doc2bow(text) for text in df["tokens"]]

    # TF-IDF
    tfidf_model = models.TfidfModel(corpus)
    corpus_tfidf = tfidf_model[corpus]
    index_tfidf = similarities.MatrixSimilarity(corpus_tfidf, num_features=len(dictionary))

    return df, dictionary, tfidf_model, index_tfidf

# ---------- STEP 3: RECOMMENDATION FUNCTION (Updated) ----------

def recommend_similar(input_text_or_company, df, dictionary, tfidf_model, index_tfidf, top_k=5):
    # Kiểm tra có trùng tên công ty không
    matched = df[df["Company Name"].str.lower() == input_text_or_company.lower()]
    if not matched.empty:
        idx = matched.index[0]
        query_vec = tfidf_model[dictionary.doc2bow(df["tokens"].iloc[idx])]
        skip_first = True
        print(f"🔍 Input is company name match: {df['Company Name'].iloc[idx]}")
    else:
        # Nếu là mô tả → xử lý như 'full_text'
        clean_input = clean_text(input_text_or_company)
        tokenized = list(dict.fromkeys(clean_input.split()))  # loại trùng từ
        query_vec = tfidf_model[dictionary.doc2bow(tokenized)]
        skip_first = False
        print(f"🔍 Input is raw description (not company name)")

    # Tính similarity
    sims = index_tfidf[query_vec]
    sorted_sims = sorted(enumerate(sims), key=lambda x: -x[1])

    # Bỏ chính công ty đó nếu là tên công ty
    start_idx = 1 if skip_first else 0

    # Trả kết quả top K
    print(f"\n📌 Top {top_k} similar companies:")
    for idx, score in sorted_sims[start_idx:start_idx + top_k]:
        print(f"- [{score:.4f}] {df['Company Name'].iloc[idx]} (ID: {df['id'].iloc[idx]})")


In [34]:
# Đọc file từ thư mục Data
df = pd.read_excel("Data/Overview_Companies.xlsx")

# Chạy pipeline
df, dictionary, tfidf_model, index_tfidf = prepare_tfidf_pipeline(df)

In [35]:
recommend_similar("FPT Software", df, dictionary, tfidf_model, index_tfidf)


🔍 Input is company name match: FPT Software

📌 Top 5 similar companies:
- [0.1036] Ingenico Vietnam (ID: 229)
- [0.0975] Collaboration Betters The World (CBTW) (ID: 76)
- [0.0936] BAN VIEN CORPORATION (ID: 38)
- [0.0906] Nami Technology Joint Stock Company (ID: 304)
- [0.0881] NTT DATA VDS (ID: 320)


In [36]:
recommend_similar("We specialize in AI and machine learning", df, dictionary, tfidf_model, index_tfidf)


🔍 Input is raw description (not company name)

📌 Top 5 similar companies:
- [0.1318] CNV Holdings (ID: 69)
- [0.1291] Viettel Post (A Member of Viettel Group) (ID: 453)
- [0.1200] K&M Holdings (ID: 251)
- [0.1113] CDEX (ID: 59)
- [0.1111] 1test (ID: 1)


In [37]:
def interactive_recommend(df, dictionary, tfidf_model, index_tfidf, top_k=5):
    while True:
        input_text = input("\n🔎 Nhập tên công ty hoặc mô tả bằng tiếng anh (hoặc gõ 'exit' để thoát): ").strip()
        if input_text.lower() == "exit":
            print("👋 Kết thúc.")
            break
        recommend_similar(input_text, df, dictionary, tfidf_model, index_tfidf, top_k=top_k)


In [38]:
interactive_recommend(df, dictionary, tfidf_model, index_tfidf)



🔎 Nhập tên công ty hoặc mô tả bằng tiếng anh (hoặc gõ 'exit' để thoát): FPT Software
🔍 Input is company name match: FPT Software

📌 Top 5 similar companies:
- [0.1036] Ingenico Vietnam (ID: 229)
- [0.0975] Collaboration Betters The World (CBTW) (ID: 76)
- [0.0936] BAN VIEN CORPORATION (ID: 38)
- [0.0906] Nami Technology Joint Stock Company (ID: 304)
- [0.0881] NTT DATA VDS (ID: 320)
👋 Kết thúc.
